In [1]:
import datetime
import numpy as np

# Function to calculate the number of days between two dates
def days_between(d1, d2):
    return (d2 - d1).days

#Accrued interest function

In [2]:
# Function to calculate accrued interest
def accrued_interest(recent_coupon_date, current_date, coupon_rate, face_value, coupon_frequency):
    days_since_coupon = days_between(recent_coupon_date, current_date)
    days_in_period = 365 / coupon_frequency
    return (days_since_coupon / days_in_period) * (coupon_rate * face_value / coupon_frequency)


#ytm computing function using newton raphson

In [5]:
def ytm_newton_raphson(pd, coupon_rate, coupon_frequency, face_value, maturity_date, current_date, recent_coupon_date, iterations=10):
    # Convert dates to datetime objects
    maturity_date = datetime.datetime.strptime(maturity_date, "%d-%m-%Y")
    current_date = datetime.datetime.strptime(current_date, "%d-%m-%Y")
    recent_coupon_date = datetime.datetime.strptime(recent_coupon_date, "%d-%m-%Y")
    
    # Calculate accrued interest
    ai = accrued_interest(recent_coupon_date, current_date, coupon_rate, face_value, coupon_frequency)
    
    # Clean price
    pc = pd - ai
    
    # Initial guess for YTM
    ytm_guess = coupon_rate
    
    # Newton-Raphson iteration
    for _ in range(iterations):
        # Calculate the price and the derivative of the price with respect to ytm
        price = 0
        derivative = 0
        for i in range(1, int((maturity_date - recent_coupon_date).days * coupon_frequency / 365) + 1):
            coupon_date = recent_coupon_date + datetime.timedelta(days=i * 365 / coupon_frequency)
            t = days_between(current_date, coupon_date) / 365.0
            discount_factor = (1 + ytm_guess / coupon_frequency) ** (coupon_frequency * t)
            price += (coupon_rate * face_value / coupon_frequency) / discount_factor
            derivative += -(t * (coupon_rate * face_value / coupon_frequency) / discount_factor) / (1 + ytm_guess / coupon_frequency)
        
        # Add the face value at maturity
        t = days_between(current_date, maturity_date) / 365.0
        discount_factor = (1 + ytm_guess / coupon_frequency) ** (coupon_frequency * t)
        price += face_value / discount_factor
        derivative += -(t * (face_value / discount_factor)) / (1 + ytm_guess / coupon_frequency)
        
        # Calculate the new guess for YTM
        ytm_guess -= (price - pc) / derivative
    
    return ytm_guess

#example 3.5

In [7]:
# Example input (Problem 3.4 and 3.5 in Accrued Interest doc)
pd = 1098  # Last traded (dirty) price
coupon_rate = 0.1  # Coupon rate
coupon_frequency = 1  # Coupon frequency (semi-annual)
face_value = 1000  # Face value
maturity_date = "31-12-2023"  # Maturity date
current_date = "01-05-2022"  # Current date
recent_coupon_date = "01-01-2020"  # Recent past coupon payment date

ytm = ytm_newton_raphson(pd, coupon_rate, coupon_frequency, face_value, maturity_date, current_date, recent_coupon_date)
print(f"Yield to Maturity: {ytm:.6f}")

Yield to Maturity: 0.639725
